In [1]:
import os   

PATH_DUUI = "data/duui-uima"
FILTER_FILES = {".py", ".ipynb"}
count = 0
LIST_XML_FILES = []
for root, subdirs, files in os.walk(PATH_DUUI):
    for file in files:
        current_file = os.path.join(root, file)
        # filter files and ignore pom.xml
        if (file.endswith(tuple(FILTER_FILES)) and (file != "pom.xml")):
            LIST_XML_FILES.append(current_file)
print(len(LIST_XML_FILES))

349


In [2]:
import sys
print(sys.executable)

/usr/bin/python3


In [3]:
#import sys
#!"{sys.executable}" -m pip install -U chromadb

In [33]:
!pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
import chromadb as cdb

client = cdb.PersistentClient("chroma")
client.delete_collection(name="DUUI_RAG_PYTHON")
collection = client.get_or_create_collection(name="DUUI_RAG_PYTHON")
print("Collection created!")

Collection created!


In [22]:
# Testing multiple ways to Embed Code
# Load model directly


In [18]:
# chunk the files with chunker

from tqdm import tqdm
import chunker
import uuid


# cast files to Chroma format

count = 0
for file in tqdm(LIST_XML_FILES):
    chunks = chunker.chunk_python_file(file, include_header=True, include_methods=True)
    ids, metas, docs = [], [], []
    for chunk in chunks:
        chroma_format = chunker.chunk_to_chroma_item(chunk.text, chunk.meta)
        # some empty documents exist
        if chroma_format["document"] != "":           
            ids.append(chroma_format["id"])
            metas.append(chroma_format["metadata"])
            docs.append(chroma_format["document"])
        else:
            print("leerer eintrag")
    if ids:
        collection.add(ids=ids, metadatas=metas, documents=docs)
count

  3%|████▌                                                                                                                                            | 11/349 [00:09<06:19,  1.12s/it]

leerer eintrag


 18%|██████████████████████████▏                                                                                                                      | 63/349 [00:52<03:26,  1.39it/s]

leerer eintrag


 19%|███████████████████████████▊                                                                                                                     | 67/349 [00:53<01:50,  2.56it/s]

...

leerer eintrag


 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                 | 307/349 [02:56<00:22,  1.88it/s]

leerer eintrag


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 349/349 [03:30<00:00,  1.66it/s]


0

In [35]:
import python_helper_api

collection.query(query_texts= """classifier = load_model(model_name)
        results = classifier.hate_prediction(texts)
        for c, res in enumerate(results):""", n_results=5)

print(python_helper_api.python_fix_code("Wie ist das DUUI-Hate-API im duui_hate.py aufgebaut (Endpoints /v1/process, /v1/typesystem, Request/Response-Modelle) und wie rufe ich es mit einem Beispiel-Request an?"))

ImportError: cannot import name 'OpenAI' from 'openai' (/home/nev/.local/lib/python3.10/site-packages/openai/__init__.py)